In [7]:
import random
import pandas as pd
from faker import Faker
import pyodbc

In [2]:
fake = Faker()

In [3]:

# Step 1: FMCG departments and categories
departments = {
    "Beverages": ["Soft Drinks", "Juices", "Bottled Water", "Tea & Coffee"],
    "Snacks & Confectionery": ["Chips", "Biscuits", "Chocolates", "Nuts & Dry Fruits"],
    "Dairy Products": ["Milk", "Cheese", "Yogurt", "Butter"],
    "Personal Care": ["Shampoo", "Soap", "Deodorants", "Toothpaste"],
    "Household Cleaning": ["Detergents", "Disinfectants", "Dishwashing", "Air Fresheners"],
    "Health & Wellness": ["Supplements", "Vitamins", "First Aid", "Sanitary Products"],
    "Packaged Foods": ["Instant Noodles", "Ready Meals", "Canned Goods", "Cooking Pastes"],
    "Baby Care": ["Diapers", "Baby Food", "Baby Lotion", "Wipes"],
    "Pet Care": ["Pet Food", "Pet Shampoo", "Pet Toys"],
    "Stationery & Supplies": ["Pens", "Notebooks", "Glue", "Markers"]
}

In [4]:

# Step 2: Generate the hierarchy
records = []
item_id_counter = 10000  # Starting item ID

for dept, categories in departments.items():
    for cat in categories:
        num_items = random.randint(10, 20)
        for _ in range(num_items):
            item_id = f"ITM{item_id_counter}"
            item_name = fake.unique.word().capitalize() + " " + cat[:-1]  # e.g., "Fresh Biscuit"
            records.append({
                "department": dept,
                "category": cat,
                "item_id": item_id,
                "item_name": item_name
            })
            item_id_counter += 1

# Step 3: Create DataFrame and save
df = pd.DataFrame(records)
# df.to_csv("product_hierarchy.csv", index=False)

# print(f"Generated {len(df)} items across departments.")


In [5]:
df

,department,category,item_id,item_name
0,Beverages,Soft Drinks,ITM10000,Leader Soft Drink
1,Beverages,Soft Drinks,ITM10001,Agreement Soft Drink
2,Beverages,Soft Drinks,ITM10002,Ground Soft Drink
3,Beverages,Soft Drinks,ITM10003,Hair Soft Drink
4,Beverages,Soft Drinks,ITM10004,Amount Soft Drink
...,...,...,...,...
540,Stationery & Supplies,Markers,ITM10540,Appear Marker
541,Stationery & Supplies,Markers,ITM10541,As Marker
542,Stationery & Supplies,Markers,ITM10542,Because Marker
543,Stationery & Supplies,Markers,ITM10543,Ask Marker


In [12]:
#  SQL Server connection details
server = 'localhost'        # Change if using a named instance e.g., 'localhost\SQLEXPRESS'
database = 'dev_FMCG_db'        # Database to create/use
username = 'sa'             # SQL Server username
password = 'sqlserver@Trupt1725' # SQL Server password

In [13]:
# Connect to SQL Server (master DB to create a new DB)
conn_master = pyodbc.connect(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};UID={username};PWD={password}'
)
conn_master.autocommit = True
cursor_master = conn_master.cursor()

In [14]:
# Create database if not exists
cursor_master.execute(f"IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = '{database}') CREATE DATABASE {database}")
print(f"Database '{database}' ready.")

cursor_master.close()
conn_master.close()

Database 'dev_FMCG_db' ready.


In [15]:
# Step 3: Connect to target DB and create table
conn = pyodbc.connect(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
)
cursor = conn.cursor()

In [ ]:
# Create table if not exists
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='product_hierarchy' AND xtype='U')
CREATE TABLE product_hierarchy (
    department NVARCHAR(100),
    category NVARCHAR(100),
    item_id NVARCHAR(20) PRIMARY KEY,
    item_name NVARCHAR(200)
)
""")
conn.commit()

In [ ]:
# Step 4: Insert data
for row in records:
    cursor.execute("""
    INSERT INTO product_hierarchy (department, category, item_id, item_name)
    VALUES (?, ?, ?, ?)
    """, row["department"], row["category"], row["item_id"], row["item_name"])

conn.commit()
cursor.close()
conn.close()

{'department': 'Beverages', 'category': 'Soft Drinks', 'item_id': 'ITM10000', 'item_name': 'Leader Soft Drink'}
{'department': 'Beverages', 'category': 'Soft Drinks', 'item_id': 'ITM10001', 'item_name': 'Agreement Soft Drink'}
{'department': 'Beverages', 'category': 'Soft Drinks', 'item_id': 'ITM10002', 'item_name': 'Ground Soft Drink'}
{'department': 'Beverages', 'category': 'Soft Drinks', 'item_id': 'ITM10003', 'item_name': 'Hair Soft Drink'}
{'department': 'Beverages', 'category': 'Soft Drinks', 'item_id': 'ITM10004', 'item_name': 'Amount Soft Drink'}
{'department': 'Beverages', 'category': 'Soft Drinks', 'item_id': 'ITM10005', 'item_name': 'Raise Soft Drink'}
{'department': 'Beverages', 'category': 'Soft Drinks', 'item_id': 'ITM10006', 'item_name': 'Thought Soft Drink'}
{'department': 'Beverages', 'category': 'Soft Drinks', 'item_id': 'ITM10007', 'item_name': 'Task Soft Drink'}
{'department': 'Beverages', 'category': 'Soft Drinks', 'item_id': 'ITM10008', 'item_name': 'Agency Soft D

In [25]:

print(f"Inserted {len(df)} rows into SQL Server table 'product_hierarchy'.")

Inserted 545 rows into SQL Server table 'product_hierarchy'.
